In [ ]:
from pymilvus import MilvusClient
from dotenv import load_dotenv
import os

load_dotenv()
# https://milvus.io/docs/quickstart.md

client = MilvusClient(
    uri=os.getenv("MILVUS_URI"),
    user=os.getenv("MILVUS_USER"),
    password=os.getenv("MILVUS_PWD"),
)

/Users/wangjiahan/mycode/python/learn-milvus/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from pymilvus import DataType

# create schema
schema = client.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

schema.add_field(field_name="my_id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="my_vector", datatype=DataType.FLOAT16_VECTOR, dim=5)
schema.add_field(field_name="my_varchar", datatype=DataType.VARCHAR, max_length=512)

{'auto_id': False, 'description': '', 'fields': [{'name': 'my_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'my_vector', 'description': '', 'type': <DataType.FLOAT16_VECTOR: 102>, 'params': {'dim': 5}}, {'name': 'my_varchar', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'enable_dynamic_field': True}

In [7]:
# set index parameters

index_params = client.prepare_index_params()

index_params.add_index(
    field_name="my_id",
    index_type="AUTOINDEX"
)

index_params.add_index(
    field_name="my_vector",
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

In [8]:
# create a collection
client.create_collection(
    collection_name="customized_setup_1",
    schema=schema,
    index_params=index_params
)

res = client.get_load_state(
    collection_name="customized_setup_1"
)

print(res)

{'state': <LoadState: Loaded>}


In [12]:
if client.has_collection(collection_name="customized_setup_3"):
    client.drop_collection(collection_name="customized_setup_3")

# set collection properties
client.create_collection(
    collection_name="customized_setup_3",
    schema=schema,
    index_params=index_params,
    num_shards=1, # sharding
    enable_mmap=True, # mmap
    properties={
        "collecton.ttl.seconds": 86400 # ttl
    },
    consistency_level="Bounded", # consistency level
)

res = client.get_load_state(
    collection_name="customized_setup_1"
)

print(res)

{'state': <LoadState: Loaded>}


In [13]:
# list collection
res = client.list_collections()
print(res)

['demo_collection', 'customized_setup_3', 'customized_setup_1']


In [14]:
# describe collection
res = client.describe_collection(collection_name="customized_setup_3")
print(res)

{'collection_name': 'customized_setup_3', 'auto_id': False, 'num_shards': 1, 'description': '', 'fields': [{'field_id': 100, 'name': 'my_id', 'description': '', 'type': <DataType.INT64: 5>, 'params': {}, 'is_primary': True}, {'field_id': 101, 'name': 'my_vector', 'description': '', 'type': <DataType.FLOAT16_VECTOR: 102>, 'params': {'dim': 5}}, {'field_id': 102, 'name': 'my_varchar', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'functions': [], 'aliases': [], 'collection_id': 458312171581037743, 'consistency_level': 2, 'properties': {'collecton.ttl.seconds': '86400'}, 'num_partitions': 1, 'enable_dynamic_field': True, 'created_timestamp': 458319723296194563}
